In [181]:
import numpy as np
import numpy as np
from gtda.time_series import SlidingWindow
from gtda.diagrams import PersistenceLandscape
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
# !pip install spectrum 
# from spectrum.periodogram import speriodogram
import plotly.express as px

In [5]:
nasdaqdatalink.read_key(filename='a5qDDyVeGxVcisfgi4q4')

In [13]:
countrynames = pd.read_csv(r"C:\Users\brian.livian\Downloads\economist_country_codes.csv")

In [89]:
countries = countrynames['COUNTRY|CODE'].apply(lambda x: x.split('|')[0])
countries = countries.apply(lambda x: x.replace(' ', ''))
codes = countrynames['COUNTRY|CODE'].apply(lambda x: x.split('|')[1])
# Dropping USA, as it will be the initial dataframe
countries = list(countries.drop(42))
codes = list(codes.drop(42))

In [140]:
globals()['UnitedStates'] =  nasdaqdatalink.get('ECONOMIST/BIGMAC_USA')
UnitedStates['UnitedStates'] = UnitedStates['dollar_price']
data = pd.DataFrame(UnitedStates['UnitedStates'])
data

,UnitedStates
Date,
2000-04-30,2.510000
2001-04-30,2.540000
2002-04-30,2.490000
2003-04-30,2.710000
2004-05-31,2.900000
2005-06-30,3.060000
2006-01-31,3.150000
2006-05-31,3.100000
2007-01-31,3.220000


In [141]:
for i in zip(countries,codes):
    country = i[0]
    code = i[1]
    df = nasdaqdatalink.get('ECONOMIST/BIGMAC_' + code)
    
    df[country] = df['dollar_price']
    df = pd.DataFrame(df[country])

    data = data.join(df)

LimitExceededError: (Status 429) (Nasdaq Data Link Error QELx04) You have exceeded the API speed limit of 20 calls per 10 minutes. Please slow down your requests.

In [145]:
# data = data.replace(0, np.nan)
# data = data.dropna(axis = 1)

In [162]:
dataPct = data.pct_change()
dataPct = dataPct.dropna()

In [189]:
figure(figsize = (20,12))
fig = px.line(data,
              width=1000, height=700,
             title = 'Big Mac')
fig.show()

<Figure size 1440x864 with 0 Axes>

In [190]:
figure(figsize = (20,12))
fig = px.line(dataPct,
              width=1000, height=700,
             title = 'Big Mac Pct')
fig.show()

<Figure size 1440x864 with 0 Axes>

In [200]:
window_size = 8
stride = 1
df = dataPct
X = df
y = df.index
SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X, y)

In [201]:
# Plot the persistence diagram and landscape for a random point cloud sliding window
from gtda.homology import VietorisRipsPersistence
pointcloud = X_sw[25]

vrp = VietorisRipsPersistence()
vrp.fit_transform_plot(pointcloud.reshape(1, *pointcloud.shape))
plt.show()

pl = PersistenceLandscape()
persistencediagram = vrp.fit_transform(pointcloud.reshape(1, *pointcloud.shape))
landscapedata = pl.fit_transform(persistencediagram)
pl.plot(landscapedata, homology_dimensions = [1], plotly_params=None)

In [202]:
# Functions to compute the Lp norms
# Find the range of x values from the persistence diagram:
def Ftseq(diagram):
    births =[]
    deaths =[]
    for pair in diagram:
        if pair[2] == 1:
            births.append(pair[0])
            deaths.append(pair[1])
    return np.linspace(min(births), max(deaths), 100)

# Calculate Lp norm:
def Lpnorm(tseq, landscapevalues, p = 1):
    norms = []
    if p.lower() == 'auc':
        for point in zip(tseq,landscapevalues):
            norms.append(np.trapz(landscapevalues, tseq))
    else: 
        for point in zip(tseq,landscapevalues):
            norms.append(np.linalg.norm(point, p))        
    return sum(norms)

In [203]:
# Calculate the norms for each of the windows in the multivariate sliding window

# Initialize empty list
Norms = np.empty(window_size)
Norms[:] = np.nan
Norms = list(Norms)
vrp = VietorisRipsPersistence()
pl = PersistenceLandscape()
for pointcloud in X_sw:
    persistencediagram = vrp.fit_transform(pointcloud.reshape(1, *pointcloud.shape))
    landscapedata = pl.fit_transform(persistencediagram)
    tseq = Ftseq(persistencediagram[0])
    yvalues = landscapedata[0][1]
    Norms.append(Lpnorm(tseq, yvalues, p = 'auc'))

In [204]:
scaler = MinMaxScaler()
data['Norms'] = Norms
data['Norms Normalized'] = scaler.fit_transform(np.array(data['Norms']).reshape(-1, 1)).reshape(-1)
data

,UnitedStates,Argentina,Australia,Brazil,Britain,Canada,Chile,China,CzechRepublic,Denmark,Euroarea,HongKong,Hungary,Indonesia,Norms,Norms Normalized
Date,,,,,,,,,,,,,,,,
2000-04-30,2.510000,2.500000,1.540000,1.650000,3.000000,1.940000,2.450000,1.200000,1.390000,3.080000,2.380000,1.310000,1.210000,1.830000,NaN,NaN
2001-04-30,2.540000,2.500000,1.520000,1.640000,2.850000,2.140000,2.100000,1.200000,1.430000,2.930000,2.270000,1.370000,1.320000,1.350000,NaN,NaN
2002-04-30,2.490000,0.780000,1.620000,1.550000,2.880000,2.120000,2.160000,1.270000,1.660000,2.960000,2.370000,1.400000,1.690000,1.710000,NaN,NaN
2003-04-30,2.710000,1.400000,1.800000,1.440000,3.080000,2.170000,1.950000,1.200000,1.910000,3.990000,2.980000,1.470000,2.140000,1.810000,NaN,NaN
2004-05-31,2.900000,1.480000,2.270000,1.700000,3.370000,2.330000,2.180000,1.260000,2.130000,4.460000,3.280000,1.540000,2.520000,1.770000,NaN,NaN
2005-06-30,3.060000,1.639627,2.497119,2.393703,3.440400,2.625260,2.531859,1.268668,2.299019,4.575659,3.583132,1.542278,2.602594,1.529597,NaN,NaN
2006-01-31,3.150000,1.550362,2.442140,2.741543,3.316132,3.007519,2.984218,1.302067,2.600944,4.492545,3.513825,1.548147,2.705889,1.543340,NaN,NaN
2006-05-31,3.100000,2.290201,2.438969,2.777175,3.647976,3.141595,2.943674,1.308248,2.671003,4.769475,3.766916,1.547728,2.713909,1.565684,NaN,NaN
2007-01-31,3.220000,2.670983,2.723258,2.999766,3.833731,3.089756,3.091734,1.414918,2.423126,4.823528,3.809211,1.538767,3.034609,1.749175,0.000000,0.000000


In [205]:
figure(figsize = (20,12))
fig = px.line(data,
              width=1000, height=700,
             title = 'Big Mac')
fig.show()

<Figure size 1440x864 with 0 Axes>